# Aligned Re-ID Optional

In [1]:
# from aligned_reid.utils.utils import load_ckpt
# import torch
# from aligned_reid.utils.utils import load_state_dict
# from aligned_reid.model.Model import Model

In [2]:
# model_path = 'model_data/run1/model_weight.pth'
# model = Model()


# # map_location = (lambda storage, loc: storage)
# # sd = torch.load(model_path, map_location=map_location)
# # load_state_dict(model, sd)
# # print('Loaded model weights from {}'.format(model_path))


# Person Re-ID Baseline

In [3]:
from model import ft_net_dense
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

In [4]:
densenet_model_path = 'model_data/model/ft_net_dense/net_last.pth'

In [5]:
use_gpu = torch.cuda.is_available()
use_gpu

True

In [6]:
def load_network(network, model_path):
    network.load_state_dict(torch.load(model_path))
    return network

In [7]:
def fliplr(img):
    inv_idx = torch.arange(img.size(3)-1,-1,-1).long()  # N x C x H x W
    img_flip = img.index_select(3,inv_idx)
    return img_flip

In [8]:
n_classes = 751 #market 151 class
model_structure = ft_net_dense(n_classes)
model_structure.load_state_dict(torch.load(densenet_model_path))
model = model_structure.cuda()
model.classifier.classifier = nn.Sequential()

In [9]:
data_transforms = transforms.Compose([
    transforms.Resize((256,128), interpolation=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
image_path_dir = '../../face_data'
image_path_dir

'../../face_data'

In [11]:
image_dataset = datasets.ImageFolder(image_path_dir, data_transforms)
dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=16, shuffle=False, num_workers=16)

In [12]:
def extract_features(model, dataloader):
    features = torch.FloatTensor()
    count = 0
    for data in dataloader: # batch_size image
        img, label = data
        n, c, h, w = img.size() # number, channel, height, width
        count += n
        
        ff = torch.FloatTensor(n, 512).zero_()
        
        for i in range(2):
            if(i==1):
                img = fliplr(img)
            input_img = Variable(img.cuda())
            
            outputs = model(input_img) 
            f = outputs.data.cpu().float()
            ff = ff+f
        
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
        ff = ff.div(fnorm.expand_as(ff))

        features = torch.cat((features,ff), 0)
        print(features)
    return features

In [13]:
hehe = 'ha'

In [14]:
%time
features = extract_features(model, dataloader)
hehe = 'yo'

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
tensor([[ 0.0101,  0.0612,  0.0000,  ...,  0.0527,  0.0000,  0.0000],
        [ 0.0371,  0.0000,  0.0000,  ..., -0.0044,  0.0449, -0.0328],
        [-0.0331, -0.0598,  0.0000,  ...,  0.0817, -0.0726, -0.0312],
        ...,
        [-0.0112,  0.0000,  0.0000,  ...,  0.0159, -0.0753,  0.0000],
        [-0.0232,  0.0000, -0.1559,  ..., -0.0853,  0.0000,  0.0180],
        [ 0.0073, -0.0572,  0.0000,  ..., -0.0326,  0.0000,  0.0044]])
tensor([[ 0.0101,  0.0612,  0.0000,  ...,  0.0527,  0.0000,  0.0000],
        [ 0.0371,  0.0000,  0.0000,  ..., -0.0044,  0.0449, -0.0328],
        [-0.0331, -0.0598,  0.0000,  ...,  0.0817, -0.0726, -0.0312],
        ...,
        [ 0.0471,  0.0000,  0.0062,  ...,  0.0000,  0.0327,  0.0188],
        [ 0.0355,  0.0000, -0.0576,  ...,  0.0070,  0.0179,  0.0781],
        [-0.0139, -0.0043,  0.0000,  ..., -0.0246,  0.0129, -0.0515]])


In [15]:
features

tensor([[ 0.0101,  0.0612,  0.0000,  ...,  0.0527,  0.0000,  0.0000],
        [ 0.0371,  0.0000,  0.0000,  ..., -0.0044,  0.0449, -0.0328],
        [-0.0331, -0.0598,  0.0000,  ...,  0.0817, -0.0726, -0.0312],
        ...,
        [ 0.0471,  0.0000,  0.0062,  ...,  0.0000,  0.0327,  0.0188],
        [ 0.0355,  0.0000, -0.0576,  ...,  0.0070,  0.0179,  0.0781],
        [-0.0139, -0.0043,  0.0000,  ..., -0.0246,  0.0129, -0.0515]])

In [16]:
from scipy.spatial.distance import euclidean

In [17]:
base_feature = features[0]
for feature in features[1:]:
    distance = euclidean(feature, base_feature)
    print(distance)

1.3162258863449097
1.5001633167266846
1.4585824012756348
1.4585890769958496
1.4806320667266846
1.4065536260604858
1.5121769905090332
1.4132683277130127
1.468139886856079
1.3985812664031982
1.4092501401901245
1.412135362625122
1.4351273775100708
1.5053290128707886
1.4828429222106934
1.3329904079437256
1.4634788036346436
1.4576482772827148
1.5322898626327515
1.408444881439209
1.3793210983276367
1.3668891191482544
1.3757606744766235
1.4014577865600586
